In [1]:
import time
import random
import argparse
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from utils import *
from model import *
from process import *

In [2]:
adj, features, labels,idx_train,idx_val,idx_test = load_citation('cora')
data='cora'
#splitstr = 'splits/'+data+'_split_0.6_0.2_'+str(0)+'.npz'
#adj, features, labels, idx_train, idx_val, idx_test, num_features, num_labels = full_load_data('cora',splitstr)
print(adj.shape)
print(features.shape)
print(labels.shape)
print(idx_train.shape)
print(idx_val.shape)
print(idx_test.shape)

C:\Users\ayday\Desktop\Master_Thesis\experiment\utils.py:87: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))


torch.Size([2708, 2708])
torch.Size([2708, 1433])
torch.Size([2708])
torch.Size([140])
torch.Size([500])
torch.Size([1000])


In [3]:
cudaid = "cuda"
device = torch.device(cudaid)
features = features.to(device)
adj = adj.to(device).coalesce()

In [4]:
feature_size=features.shape[1]
num_classes = len(torch.unique(labels))
hidden_dim=16
dropout=0.5
model=GCN(feature_size,hidden_dim,num_classes,dropout)
model=model.to(device)
optimizer = optim.Adam(model.parameters(),lr=0.01,weight_decay=5e-4)

epochs=100
patience=10
test=False

In [9]:
def train():
    model.train()
    optimizer.zero_grad()
    output = model(features,adj)
    acc_train = accuracy(output[idx_train], labels[idx_train].to(device))
    loss_train = F.nll_loss(output[idx_train], labels[idx_train].to(device))
    loss_train.backward()
    optimizer.step()
    return loss_train.item(),acc_train.item()


def validate():
    model.eval()
    with torch.no_grad():
        output = model(features,adj)
        loss_val = F.nll_loss(output[idx_val], labels[idx_val].to(device))
        acc_val = accuracy(output[idx_val], labels[idx_val].to(device))
        return loss_val.item(),acc_val.item()

def test():
    #model.load_state_dict(torch.load(checkpt_file))
    model.eval()
    with torch.no_grad():
        output = model(features, adj)
        loss_test = F.nll_loss(output[idx_test], labels[idx_test].to(device))
        acc_test = accuracy(output[idx_test], labels[idx_test].to(device))
        return loss_test.item(),acc_test.item()

'''
t_total = time.time()
bad_counter = 0
best = 999999999
best_epoch = 0
acc = 0
for epoch in range(epochs):
    loss_tra,acc_tra = train()
    loss_val,acc_val = validate()
    if(epoch+1)%1 == 0: 
        print('Epoch:{:04d}'.format(epoch+1),
            'train',
            'loss:{:.3f}'.format(loss_tra),
            'acc:{:.2f}'.format(acc_tra*100),
            '| val',
            'loss:{:.3f}'.format(loss_val),
            'acc:{:.2f}'.format(acc_val*100))
    if loss_val < best:
        best = loss_val
        best_epoch = epoch
        acc = acc_val
        #torch.save(model.state_dict(), checkpt_file)
        bad_counter = 0
    else:
        bad_counter += 1

    if bad_counter == patience:
        break

if test:
    acc = test()[1]

print("Train cost: {:.4f}s".format(time.time() - t_total))
print('Load {}th epoch'.format(best_epoch))
print("Val","acc.:{:.1f}".format(acc*100))
'''

'\nt_total = time.time()\nbad_counter = 0\nbest = 999999999\nbest_epoch = 0\nacc = 0\nfor epoch in range(epochs):\n    loss_tra,acc_tra = train()\n    loss_val,acc_val = validate()\n    if(epoch+1)%1 == 0: \n        print(\'Epoch:{:04d}\'.format(epoch+1),\n            \'train\',\n            \'loss:{:.3f}\'.format(loss_tra),\n            \'acc:{:.2f}\'.format(acc_tra*100),\n            \'| val\',\n            \'loss:{:.3f}\'.format(loss_val),\n            \'acc:{:.2f}\'.format(acc_val*100))\n    if loss_val < best:\n        best = loss_val\n        best_epoch = epoch\n        acc = acc_val\n        #torch.save(model.state_dict(), checkpt_file)\n        bad_counter = 0\n    else:\n        bad_counter += 1\n\n    if bad_counter == patience:\n        break\n\nif test:\n    acc = test()[1]\n\nprint("Train cost: {:.4f}s".format(time.time() - t_total))\nprint(\'Load {}th epoch\'.format(best_epoch))\nprint("Val","acc.:{:.1f}".format(acc*100))\n'

In [13]:
epochs = 100  # Set the number of epochs
patience = 10  # Set the patience for early stopping
Test = True  # Flag for testing after training

# Initialize lists to store accuracy for each run
val_acc_list = []

# Run the training process 10 times
for run in range(10):
    t_total = time.time()
    bad_counter = 0
    best = 999999999
    best_epoch = 0
    acc = 0
    for epoch in range(epochs):
        loss_tra, acc_tra = train()
        loss_val, acc_val = validate()
        if (epoch + 1) % 1 == 0:
            '''
            print('Run:{:02d}'.format(run+1),
                  'Epoch:{:04d}'.format(epoch+1),
                  'train',
                  'loss:{:.3f}'.format(loss_tra),
                  'acc:{:.2f}'.format(acc_tra * 100),
                  '| val',
                  'loss:{:.3f}'.format(loss_val),
                  'acc:{:.2f}'.format(acc_val * 100))
            '''
        if loss_val < best:
            best = loss_val
            best_epoch = epoch
            acc = acc_val
            # torch.save(model.state_dict(), checkpt_file)
            bad_counter = 0
        else:
            bad_counter += 1

        if bad_counter == patience:
            break

  
    acc = test()[1]

    val_acc_list.append(acc)

    #print("Train cost: {:.4f}s".format(time.time() - t_total))
    #print('Load {}th epoch'.format(best_epoch))
    #print("Val", "acc.:{:.1f}".format(acc * 100))

# Calculate mean and standard deviation of validation accuracy
val_acc_array = np.array(val_acc_list)
mean_acc = val_acc_array.mean()
std_acc = val_acc_array.std()

print("Mean validation accuracy: {:.2f}%".format(mean_acc * 100))
print("Standard deviation of validation accuracy: {:.2f}%".format(std_acc * 100))

Mean validation accuracy: 81.32%
Standard deviation of validation accuracy: 0.47%
